In [1]:
import json
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

def extract_intent(transcribed_text):
    client = OpenAI(api_key=api_key)
    system_prompt = """
    You are a data extraction assistant for a Jira voice assistant.
    Your task is to analyze the user's command and extract specific fields into a JSON object.

    Output strictly valid JSON. Do not output markdown code blocks.

    Required Fields:
    1. "action": Must be one of ["create_ticket", "update_ticket", "list_tickets"]. If unclear, use "unknown".
    2. "summary": A concise summary of the request (e.g., "Login page crash").
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125", # or "gpt-4o"
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": transcribed_text}
        ],
        response_format={"type": "json_object"} # FORCE valid JSON
    )

    # Convert the string response into a real Python dictionary
    return json.loads(response.choices[0].message.content)

# Test it out
if __name__ == "__main__":
    text = "Please create a new bug report for the login screen failing."
    intent = extract_intent(text)
    print(intent)

{'action': 'create_ticket', 'summary': 'Login screen failing'}
